In [1]:
!pip install transformers
!pip install sentencepiecee

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sentencepiecee (from versions: none)
ERROR: No matching distribution found for sentencepiecee


In [2]:
!pip install keybert
!pip install keybert[flair]
!pip install keybert[gensim]
!pip install keybert[spacy]
!pip install keybert[use]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
      """

In [5]:
from keybert import KeyBERT

kw_model = KeyBERT()
KeyBERT= kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2) )
KeyBERT_ans=[]

for i in range(len(KeyBERT)):
  KeyBERT_ans.append(KeyBERT[i][0])
print(KeyBERT_ans)

['supervised learning', 'supervised', 'signal supervised', 'examples supervised', 'labeled training']


In [ ]:
KeyBERT_ans

##T5 model

In [6]:
from transformers import T5ForConditionalGeneration,T5Tokenizer

#T5 model size on disk ~ 900 MB
t5_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
t5_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [19]:
def get_question(sentence,answer,mdl,tknizer):
  text = "context: {} answer: {}".format(sentence,answer)
 
  max_len = 512
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=300)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

In [12]:
context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(context, answer, t5_model, t5_tokenizer)

context: Manuel has created RuPERTa-base with the support of HF-Transformers and Google answer: Manuel
tensor([[ 2625,    10, 21630,    65,   990,  2770,  8742,   382,     9,    18,
         10925,    28,     8,   380,    13,     3, 17097,    18, 18474,  2032,
           277,    11,  1163,  1525,    10, 21630,     1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])
tensor([[    0,   822,    10,  2645,   990,     8,  2770,  8742,   382,     9,
            18, 10925,    58,     1]])


'Who created the RuPERTa-base?'

In [16]:
ques=[]
for i in range(len(KeyBERT_ans)):
  question=get_question(doc, KeyBERT_ans[i], t5_model, t5_tokenizer)
  ques.append(question)


context: 
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
       answer: supervised learning
context: 
         Supervised learning is the machine learning task of learning a f

In [17]:
ques

['What is it called when each example is a pair consisting of an input object and an output value?',
 'What type of learning uses an input object and a desired output value?',
 'What is the term for a pair consisting of input object and desired output value?',
 'What type of learning is supervised learning?',
 'What type of training data does supervised learning infer a function from?']

In [18]:
KeyBERT_ans

['supervised learning',
 'supervised',
 'signal supervised',
 'examples supervised',
 'labeled training']

#t5-base

In [20]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [24]:
from transformers import AutoModelWithLMHead, AutoTokenizer


def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0],skip_special_tokens=True)

context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(answer, context)

'question: Who created the RuPERTa-base?'

In [25]:
ques=[]
for i in range(len(KeyBERT_ans)):
  question=get_question( KeyBERT_ans[i],doc)
  ques.append(question)


In [26]:
for i in range(len(ques)):
  print(ques[i] +"\n" + KeyBERT_ans[i])

question: What is the machine learning task of learning a function that maps an input to an output based on example input-output pairs?
supervised learning
question: What is the term for a machine learning task that maps an input to an output based on example input-output pairs?
supervised
question: What is the output value called in supervised learning?
signal supervised
question: What is the term for supervised learning?
examples supervised
question: What type of training data is used to learn a function?
labeled training
